In [1]:
import sys
sys.path.insert(0, '../../')
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')

from pyplr.stlab import SpectraTuneLab
from pyplr.calibrate import CalibrationContext

In [2]:
d = SpectraTuneLabScene(password='23acd0c3e4c5c533', identity=1, lighthub_ip='192.168.6.2')


NameError: name 'SpectraTuneLabScene' is not defined

In [ ]:
def interp_spectra(spectra):
    '''
    This function needs generalising.

    Parameters
    ----------
    spectra : TYPE
        DESCRIPTION.

    Returns
    -------
    intp_tbl : TYPE
        DESCRIPTION.

    '''
    #tbl = spectra.unstack(level=0)
    #tbl.columns = [val[1] for val in tbl.columns]
    
    intp_tbl = pd.DataFrame()
    for led, df in spectra.groupby(['led']):
        intensities = df.index.get_level_values('intensity')
        new_intensities = np.linspace(intensities.min(), intensities.max(), 4096)
        new_intensities = new_intensities.astype('int')
        df.reset_index(inplace=True, drop=True)
        #df.columns = range(0, df.shape[1])
        df.index = df.index * 63
        n = df.reindex(new_intensities).interpolate(method='linear')
        n['intensity'] = n.index
        n['led'] = led
        intp_tbl = intp_tbl.append(n)
    intp_tbl.set_index(['led','intensity'], inplace=True)
    return intp_tbl

In [ ]:
cc = CalibrationContext('../data/S2_corrected_oo_spectra.csv', binwidth=1)
LED3_max = 1800 # comfortable maximal melanopsin stim
r_matches = {}
for led in [1,3,5,9]:
    match = cc.match(match_led=3, match_led_intensity=LED3_max, target_led=led, match_type='irrad')
    r_matches[led] = match

idxs = [(led,intensity[1]) for led, intensity in r_matches.items()]
stim_info = cc.aopic.loc[idxs]
stim_info['lux'] = cc.lux.loc[idxs]
stim_info['irradiance'] = cc.lkp.sum(axis=1)
stim_info.rename(columns={'irradiance':'Irradiance'}, inplace=True)
stim_info

In [ ]:
cc.lkp.loc[[(led, 4095) for led in range(10)]].max(axis=1)

In [ ]:
cc.lkp.loc[[(led, 4095) for led in range(10)]].idxmax(axis=1)

In [ ]:
c = stlab.get_led_colors()
c = [c[i] for i in stim_info.index.get_level_values(0)]
d = stim_info.reset_index().melt(id_vars=['led','intensity'], 
                          value_name='flux',
                          var_name='aopic')
d = d[d.aopic!='lux']
fg = sns.catplot(x='aopic', 
                  y='flux', 
                  data=d, 
                  hue='led', 
                  kind='bar', 
                  palette=c, 
                  alpha=.8)
fg.set(xlabel='',
       ylabel='W/m$^2$')
plt.xticks(rotation=45)
fg._legend.set_title('$\lambda$-max')
fg._legend.set_bbox_to_anchor((1.1, .5))
new_labels = ['446', '475', '522', '657']
sns.despine(top=False, right=False)
for t, l in zip(fg._legend.texts, new_labels): 
    t.set_text(l)
fg.savefig('chirp_stims.tiff', dpi=300)


In [ ]:
c = stlab.get_led_colors()
c = [c[i] for i in stim_info.index.get_level_values(0)]
d = stim_info.reset_index().melt(id_vars=['led','intensity'], 
                          value_name='flux',
                          var_name='aopic')
d = d[d.aopic=='lux']
fg = sns.catplot(x='led', 
                  y='flux', 
                  data=d, 
                  kind='bar', 
                  palette=c, 
                  alpha=.8,
                legend=False)
fg.set(xlabel='',
       ylabel='Lux',
       xticklabels=new_labels)
plt.xticks(rotation=45)
sns.despine(top=False, right=False)
plt.savefig('chirp_lux.tiff', dpi=300)

In [ ]:
d

In [ ]:
d = cc.lkp.loc[idxs].reset_index().melt(id_vars=['led','intensity'], 
                                        value_name='flux',
                                        var_name='Wavelength')
fig, ax = plt.subplots()
sns.lineplot(data=d, 
             x='Wavelength', 
             y='flux', 
             units='led', 
             estimator=None, 
             hue='led', 
             palette=c, 
             legend=False, 
             ax=ax)
ax.set(ylabel='W/m$^2$/nm',
      xlabel='Wavelength (nm)')
sns.despine(top=False, right=False)
fig.savefig('chirp_SPDs.tiff', dpi=300, bbox_inches='tight')


In [ ]:
from scipy.signal import chirp

for (led, intensity), info in stim_info.iterrows():

    # stimulus params
    videoname = 'chirp_{}'.format(led)
    Fs = 100
    mintensity = 0
    maxtensity = intensity

    # stimulus params
    background  = maxtensity/2
    amplitude   = maxtensity/2
    duration = 20
    n = 2000
    t = np.linspace(0, duration, n, endpoint=False)
    f0 = 0.01
    f1 = 1

    # define frequency modulation
    chp = chirp(t, f0, duration, f1, method='linear', phi=270)
    chp = stlab.modulate_intensity_amplitude(chp, background, amplitude)
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(chp)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # define amplitude modulation
    time  = stlab.get_time_vector(duration)
    sm = stlab.sinusoid_modulation(.5, duration, Fs)
    ivals = stlab.modulate_intensity_amplitude(sm, background, np.linspace(0,amplitude,duration*Fs))
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(ivals)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time");

    duration=60
    time = stlab.get_time_vector(duration)
    stim = np.zeros(duration*Fs)

    # assemble time course
    stim[0:100]     = maxtensity # 1s pulse
    stim[800:900]   = background # step
    stim[900:2900]  = chp  # chirp
    stim[2900:3100] = background
    stim[3100:5100] = ivals
    stim[5100:5300] = background
    stim = stim.astype('int')

    # plot
    #fig, ax = plt.subplots(figsize=(16,4))
    #ax.plot(stim)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # make video file
    cols = stlab.get_video_cols()
    zs = np.zeros(duration*Fs)
    zs = zs.astype('int')
    data = [zs for x in range(10)]
    data[led] = stim

    #data = [stim for x in range(10)]
    data.insert(0, time)
    df = pd.DataFrame(data, cols).T
    setting = {'led':led,'max_intensity':intensity}
    info = {**info.to_dict(), **setting}
    stlab.make_video_file(df, videoname, **info)

In [ ]:
#stim = np.hstack([np.zeros(500),stim])
fig, ax = plt.subplots(figsize=(14,4))
ax.plot(np.arange(0, len(stim)), stim, c='k')
plt.xlabel('Time (s)')
plt.yticks([0,1322],['min','max'])
plt.xticks(range(500, 7000, 1000), [str(val) for val in range(0, 70, 10)])
sns.despine()
plt.savefig('chirp_profile.svg', bbox_inches='tight')

In [ ]:
import glob
from time import sleep

for chp in glob.glob('chirp_*.dsf'):
    d.load_video_file(chp)
    d.play_video_file()
    sleep(60)

In [ ]:
glob.glob('chirp_*.dsf')